In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

In [ ]:
from tmqrfeed.manager import DataManager
from tmqrindex.index_exo_base import IndexEXOBase
from datetime import datetime

In [ ]:
class EXODeltaTargetGeneric(IndexEXOBase):
    _description_short = "EXO Vanilla DeltaTargeting Long"
    _description_long = ""

    _index_name = "EXODeltaTargetGeneric"
    
    def calc_exo_logic(self):
        """
        Calculates SmartEXO logic.
        NOTE: this method must use self.dm.quotes() or self.dm.quotes(series_key='for_secondary_series') to 
              calculate SmartEXO logic
        :return: Pandas.DataFrame with index like in dm.quotes() (i.e. primary quotes)
        """
        pass
    
    def manage_position(self, dt, pos, logic_df):
        """
        Manages opened position (rollover checks/closing, delta hedging, etc)
        :param dt: current datetime
        :param pos: Position instance
        :param logic_df: result of calc_exo_logic()[dt]  if applicable
        :return: nothing, manages 'pos' in place
        """
        if pos.almost_expired_ratio(dt) > 0:
            pos.close(dt)

    def construct_position(self, dt, pos, logic_df):
        """
        EXO position construction method
        :param dt: current datetime
        :param pos: Position instance
        :param logic_df: result of calc_exo_logic()[dt]  if applicable
        :return: nothing, manages 'pos' in place
        """
        
        fut, opt_chain = self.dm.chains_options_get(self.instrument, dt)
        
        #
        # Help
        # 
        """
        Find option contract in chain using 'how' criteria
        :param dt: analysis date
        :param item: search value
        :param opttype: option type 'C' or 'P'
        :param how: search method
                    - 'offset' - by strike offset from ATM
                    - 'strike' - by strike absolute value
                    - 'delta'  - by delta
                        Search option contract by delta value:
                        If delta ==  0.5 - returns ATM call/put
                        If delta > 0.5 - returns ITM call/put near target delta
                        If delta < 0.5 - returns OTM call/put near target delta
        :param kwargs:
            * how == 'offset' kwargs:
                - error_limit - how many QuoteNotFound errors occurred before raising exception (default: 5)
            * how == 'delta' kwargs:
                - error_limit - how many QuoteNotFound errors occurred before raising exception (default: 5)
                - strike_limit - how many strikes to analyse from ATM (default: 30)
        :return: OptionContract
        Example:
        pos.add_transaction(dt, opt_chain.find(dt, 0.15, 'C', how='delta'), 1.0)
        
        pos.add_transaction(dt, # Current date
                            opt_chain.find( # Find option in chain by delta
                                            dt,   # Current date     
                                            0.15, # Delta value (because how='delta'), otherwise ATM offset
                                            'C',  # Search for call
                                            how='delta'), # Search by delta                            
                            1.0 # Transaction Qty
                            )
        
        """        
        
        pos.add_transaction(dt, opt_chain.find(dt, 0.15, 'C', how='delta'), 1.0)
        pos.add_transaction(dt, opt_chain.find(dt, 0.25, 'P', how='delta'), -1.0)

In [ ]:
from collections import OrderedDict

d = OrderedDict()
d[1] = 'a'
d[2] = 'b'
d['c'] = 3

In [ ]:
for k,v in reversed(d.items()):
    print(k)

In [ ]:
dm = DataManager(date_start=datetime(2016, 5, 1))

In [ ]:
index = EXODeltaTargetGeneric(dm, instrument="US.ES")

In [ ]:
index.run()


In [ ]:
index.data['equity_execution'].plot()

In [ ]:
index.save()

In [ ]:
loaded_index = EXOLongCall.load(dm, instrument='US.CL')

In [ ]:
loaded_index.data['equity_execution'].plot()

In [ ]:
x = None

In [ ]:
x['X']